In [333]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [334]:
from src.features.rankings import RankingFeatures
from src.utils import load_data_template

In [335]:
data = load_data_template(season=False)
data.dropna(subset=['a_win'], inplace=True)
rank_feat = RankingFeatures(default_lags=1)
data = rank_feat.per_team_wrapper(data, rank_feat.pca_variables_rankings, per_game=False, per_day=True)
data.dropna(inplace=True)
print(data.shape)
data.sample()

(981, 87)


,Season,team_a,team_b,in_target,game_set,a_win,DayNum,PC1_team_a,PC2_team_a,PC3_team_a,...,PC11_team_b_lag-1,PC12_team_b_lag-1,PC13_team_b_lag-1,PC14_team_b_lag-1,PC15_team_b_lag-1,PC16_team_b_lag-1,PC17_team_b_lag-1,PC18_team_b_lag-1,PC19_team_b_lag-1,PC20_team_b_lag-1
10263,2003,1173,1409,False,0,False,136,-233.924592,-6.133105,-16.823198,...,-1.87504,-31.444577,30.043971,-29.867701,-26.371367,9.806716,10.088343,7.660619,-18.572966,-14.692247


In [337]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target', 'DayNum'], axis=1)
y = data[['a_win']].astype(int)

kf = KFold(n_splits=5, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

0.5690231582854265
0.6568035867126962
0.5254022798700524
0.5944301750681991
0.5863286301455631
Metric Mean: 0.59 (0.04)
